In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

In [18]:
import torch
import tiktoken
from transformers import AutoTokenizer, AutoModel, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [5]:
def created_sbert_mpnet():
    model = "sentence-transformers/all-mpnet-base-v2"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return HuggingFaceEmbeddings(model_name = model, model_kwargs= {"device" : device})

def created_flan_t5_base(load_in_8bit):
    model = "google/flan-t5-xl"
    tokenizer = AutoTokenizer.from_pretrained(model)

    return pipeline(
        "text2text-generation",
        model = model,
        tokenizer = tokenizer,
        max_new_tokens = 100,
        device_map = "auto",
        model_kwargs= {"load_in_8bit" : load_in_8bit,
                      "temperature" : 0.95}
    )

embedding = created_sbert_mpnet()
llm = created_flan_t5_base(load_in_8bit= False)

/tmp/ipykernel_30/1050973921.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name = model, model_kwargs= {"device" : device})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
_question = "The store-owner observed and listened to the conversation: The boy asked, Lady, Can you giveme the job of cutting your lawn?"

prompt_template = "Answer the following question. \n" + _question
result = llm(prompt_template)
for res in result:
    print(res['generated_text'])

The store-owner was surprised.


In [24]:
pdf_path = "https://charbzaban.com/wp-content/uploads/2018/01/Charbzaban.com_.pdf"
loader = PDFPlumberLoader(pdf_path)
documents = loader.load()

text_splitter = TokenTextSplitter(chunk_size= 100, chunk_overlap= 10)
texts = text_splitter.split_documents(documents)

persist_directory = "/output/kaggle/working/chromadb"
vectordb = Chroma.from_documents(documents = documents, embedding = embedding, persist_directory= persist_directory)

In [27]:
hf_llm  = HuggingFacePipeline(pipeline = llm)
retriever = vectordb.as_retriever(search_kwargs = {"k" : 1})
qa = RetrievalQA.from_chain_type(llm = hf_llm, chain_type = 'stuff', retriever= retriever)

question_t5_prompt = """
context : {context}
question : {question}
answer :
"""

QUESTION_T5_PROMPT = PromptTemplate(
    template = question_t5_prompt, input_variables = ['context', 'question']
)

qa.combine_documents_chain.llm_chain.prompt = QUESTION_T5_PROMPT

In [29]:
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({'query' : _question})

/tmp/ipykernel_30/3483137872.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa({'query' : _question})




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'The store-owner observed and listened to the conversation: The boy asked, Lady, Can you giveme the job of cutting your lawn?',
 'result': '[ii]',
 'source_documents': [Document(metadata={'Creator': 'Nitro Pro 9', 'ModDate': "D:20180114004334-08'00'", 'file_path': 'https://charbzaban.com/wp-content/uploads/2018/01/Charbzaban.com_.pdf', 'page': 12, 'source': 'https://charbzaban.com/wp-content/uploads/2018/01/Charbzaban.com_.pdf', 'total_pages': 71}, page_content='I was just checking my performance\nA little boy went into a drug store, reached for a soda carton and pulled it over to the telephone.\nHe climbed onto the carton so that he could reach the buttons on the phone and proceeded to\npunch in seven digits.\nThe store-owner observed and listened to the conversation: The boy asked, "Lady, Can you give\nme the job of cutting your lawn? The woman replied, "I already have someone to cut my lawn."\n"Lady, I will cut your lawn for half the price of the person who cuts your lawn 